In [1]:
import pandas as pd
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [2]:
# 데이터 불러오기 
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

In [3]:
test_df

,index,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,0,0,0,0,0,0,0,0,9,8,...,103,87,56,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,34,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,14,53,99,...,0,0,0,0,63,53,31,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,137,126,140,0,133,224,222,56,0,0
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0,0,0,0,0,0,0,0,0,...,32,23,14,20,0,0,1,0,0,0
9996,9996,0,0,0,0,0,0,0,0,0,...,0,0,0,2,52,23,28,0,0,0
9997,9997,0,0,0,0,0,0,0,0,0,...,175,172,172,182,199,222,42,0,1,0
9998,9998,0,1,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
#학습에 사용하기 위해서는 float형태로 바뀌어야 한다.
train_data = np.array(train_df.iloc[:,1:], dtype = 'float32') #인덱스를 제외하고 넣기
test_data = np.array(test_df.iloc[:,1:], dtype='float32')

In [5]:
x_train = train_data[:, 1:] # 피처들
y_train = train_data[:,0]  # 라벨

x_test = test_data

In [6]:
#validation set 두기 (정확한 평가를 위해서)
#주로 하이퍼 파라미터 찾기 위해서 validation set을 둔다.
from sklearn.model_selection import train_test_split
x_train,x_validate, y_train,y_validate = train_test_split(x_train,y_train,test_size = 0.2,random_state = 12345)

print(x_train.shape, x_validate.shape)


(48000, 784) (12000, 784)


In [7]:
# 28 X 28 이미지로 만들어준다.
# reshape를 이용해서 다시 이미지로 만들어주기.
x_train.shape #(48000, 28*28  )

x_train = x_train.reshape(-1,28,28) # 28 * 28 사이즈 이미지
x_validate = x_validate.reshape(-1,28,28) #28 *28 이미지로

x_test = x_test.reshape(-1,28,28) # 28 * 28 사이즈 이미지


In [9]:
#모델 구축
from keras.models import Sequential
from keras.layers import Dense,BatchNormalization,Dropout,Activation, Conv2D,MaxPool2D,Flatten

#28 * 28
nn_model = Sequential([
    # 1채널 -> 32 채널 (filters)
    Conv2D(filters=32,kernel_size=(3,3),input_shape=(28,28,1),padding='same'), #이미지 채널 1-> 32
    MaxPool2D(pool_size=2),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),
    


    Conv2D(filters=64,kernel_size=(3,3),input_shape=(28,28,1),padding='same'), #이미지 채널 32-> 64
    MaxPool2D(pool_size=2),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),


    Conv2D(filters=64,kernel_size=(3,3),input_shape=(28,28,1),padding='same'), #이미지 채널 64-> 64
    MaxPool2D(pool_size=2),
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Flatten(),
    Dense(512), # 576 -> 512
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Dense(64), # 512 -> 64 
    BatchNormalization(),
    Activation('relu'),
    Dropout(0.2),

    Dense(32), # 64-> 32 
    Dense(10,activation='softmax'), # 32 -> 10
])
#feature는 아니지만, 깊게 노드를 연결하면 스스로 feature를 추출할 수 있게 된다는 뜻.

# convolution filter 변경 
# dense layer


In [10]:
nn_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 batch_normalization_5 (Batc  (None, 14, 14, 32)       128       
 hNormalization)                                                 
                                                                 
 activation_5 (Activation)   (None, 14, 14, 32)        0         
                                                                 
 conv2d_4 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 7, 7, 64)        

In [14]:
nn_model( np.random.rand(1,28,28)  ).shape

# TensorShape([1, 3, 3, 64]) # 1개 이미지 ->  3 x 3 64개의 이미지
# 3x3x64 -> 576개의 숫자 (feature. 임베딩 )-> (뉴럴 네트워크)fully connected layer로 분류하자.

TensorShape([1, 3, 3, 64])

In [11]:
nn_model(np.random.rand(1,784)).shape

TensorShape([1, 10])

In [11]:
#compile이란 우리가 만든 모델을 어떻게 weight들을 최적화 시킬지 정해주는 부분.
from keras.optimizers import Adam
nn_model.compile(loss ='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=0.001),metrics =['accuracy'])
#Adam은 가중치를 학습 시켜주는 optimizer중 가장 유명한 것.
#learning rate는 데이터, 전처리 방법에 따라 달라진다. 스스로 조절해가며 찾아야한다.
#metrics는 학습하면서 평가해주는 중간결과를 보여줄 때 사용.

#sparse_categorical_crossentropy는 0~10으로된 수치형 라벨을 10개로 만들어진 백터와 매칭시켜준다.

#categorical_crossentropy를 쓰면,0~10칸으로 된 라벨을 10개 벡터와 매칭.


# 0 ~ 10 

In [12]:
nn_model.fit(
    x_train,
    y_train,
    batch_size=32,
    epochs=30,
    verbose=1,
    validation_data=(x_validate,y_validate),
)

Epoch 1/30
1500/1500 [==============================] - 14s 8ms/step - loss: 0.4053 - accuracy: 0.8545 - val_loss: 0.2823 - val_accuracy: 0.8942
Epoch 2/30
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2763 - accuracy: 0.8979 - val_loss: 0.2629 - val_accuracy: 0.8986
Epoch 3/30
1500/1500 [==============================] - 11s 7ms/step - loss: 0.2332 - accuracy: 0.9145 - val_loss: 0.2609 - val_accuracy: 0.9047
Epoch 4/30
1500/1500 [==============================] - 11s 7ms/step - loss: 0.1996 - accuracy: 0.9268 - val_loss: 0.2244 - val_accuracy: 0.9151
Epoch 5/30
1500/1500 [==============================] - 11s 7ms/step - loss: 0.1774 - accuracy: 0.9336 - val_loss: 0.2363 - val_accuracy: 0.9148
Epoch 6/30
1500/1500 [==============================] - 11s 7ms/step - loss: 0.1560 - accuracy: 0.9426 - val_loss: 0.2368 - val_accuracy: 0.9172
Epoch 7/30
1500/1500 [==============================] - 11s 7ms/step - loss: 0.1386 - accuracy: 0.9490 - val_loss: 0.2359 - val_ac

In [13]:
y_pred = nn_model.predict(x_test)

313/313 [==============================] - 1s 3ms/step


In [14]:
y_pred.shape
#10칸에 대해, 확률들을 적어 둔 것.

(10000, 10)

In [15]:
y_pred

array([[9.9184167e-01, 9.0516461e-10, 2.9971950e-05, ..., 1.7302814e-09,
        1.4212447e-04, 1.1454150e-08],
       [3.6248914e-11, 9.9999964e-01, 6.6010972e-14, ..., 4.9027892e-17,
        6.7052214e-12, 4.3936607e-16],
       [4.4364834e-04, 2.3361740e-07, 9.7490275e-01, ..., 9.8739861e-07,
        1.1246160e-06, 3.5701831e-08],
       ...,
       [3.2330141e-10, 1.2298093e-16, 1.0322961e-12, ..., 1.7749380e-10,
        1.0000000e+00, 2.3595761e-12],
       [3.1616858e-07, 1.8354594e-14, 1.4756051e-09, ..., 2.5072131e-09,
        9.9991059e-01, 7.6048384e-10],
       [9.8964851e-04, 9.8786968e-01, 2.4660392e-04, ..., 1.4462357e-07,
        1.2969100e-03, 1.3521472e-07]], dtype=float32)

In [16]:
submission = pd.read_csv('./data/sample_submission.csv', encoding = 'utf-8')
submission['label'] = y_pred.argmax(axis=1)
submission.to_csv('fashion_submission.csv', index = False)

In [17]:
submission['label'].max()

9

In [48]:

submission

,index,label
0,0,0
1,1,1
2,2,2
3,3,2
4,4,3
...,...,...
9995,9995,0
9996,9996,6
9997,9997,8
9998,9998,8


In [1]:
#이미지로 확인하기
y_pred

NameError: name 'y_pred' is not defined

점수를 높일 수 있는 시도들
1. EPOCH 변화
2. 레이어 변화
3. learning rate 변화
4. ensemble